In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 20
verbosity = 1
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 386 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/20
40/40 [==============================] - 26s 652ms/step - loss: 1.8619 - auc: 0.5237 - accuracy: 0.5230 - val_loss: 0.6611 - val_auc: 0.7572 - val_accuracy: 0.5729
Epoch 2/20
40/40 [==============================] - 25s 621ms/step - loss: 0.8439 - auc: 0.5663 - accuracy: 0.5447 - val_loss: 1.2985 - val_auc: 0.8225 - val_accuracy: 0.2005
Epoch 3/20
40/40 [==============================] - 25s 613ms/step - loss: 0.9271 - auc: 0.5876 - accuracy: 0.5557 - val_loss: 0.4098 - val_auc: 0.8418 - val_accuracy: 0.8464
Epoch 4/20
40/40 [==============================] - 25s 615ms/step - loss: 0.7787 - auc: 0.6400 - accuracy: 0.5983 - val_loss: 0.4434 - val_auc: 0.8593 - val_accuracy: 0.8620
Epoch 5/20
40/40 [==============================] - 25s 613ms/step - loss: 0.6800 - auc: 0.6930 - accuracy: 0.6314 - val_loss: 0.3490 - val_auc: 0.8736 - val_accuracy: 0.8333
Epoch 6/20
40/40 [==============================] - 25s 615ms/step - loss: 0.9448 - auc: 0.6226 - accuracy: 0.5727 - val_loss

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

283
103


In [8]:
prediction

array([[0.9307172 ],
       [0.72516495],
       [0.02693149],
       [0.7571228 ],
       [0.80002666],
       [0.6670694 ],
       [0.25526133],
       [0.35767594],
       [0.9614811 ],
       [0.87262183],
       [0.25346926],
       [0.9328381 ],
       [0.39124292],
       [0.5188827 ],
       [0.9892587 ],
       [0.906072  ],
       [0.45301503],
       [0.8654009 ],
       [0.9003533 ],
       [0.9403409 ],
       [0.29650584],
       [0.8657913 ],
       [0.8919785 ],
       [0.62354857],
       [0.551372  ],
       [0.99929965],
       [0.9966394 ],
       [0.94452   ],
       [0.09219592],
       [0.7038104 ],
       [0.96553606],
       [0.6730056 ],
       [0.8323554 ],
       [0.9949753 ],
       [0.44953534],
       [0.62772393],
       [0.7906894 ],
       [0.99269015],
       [0.9100799 ],
       [0.5124869 ],
       [0.6391865 ],
       [0.26007965],
       [0.97109365],
       [0.74593943],
       [0.19243008],
       [0.63126725],
       [0.79818666],
       [0.405

In [9]:
tf.__version__

'2.3.1'